In [1]:
import os
import time
import copy
import numpy as np
import pandas as pd
from PIL import Image
import torch,torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets,models,transforms
import torch.optim as optim
# from torchsummary import summary
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix

In [2]:
# Read train,val,test txt files
# train_df = pd.read_csv('../input/covidxct/train_COVIDx_CT-2A.txt', sep=" ", header=None)
# train_df.columns=['filename', 'label', 'xmin','ymin','xmax','ymax']

# val_df = pd.read_csv('../input/covidxct/val_COVIDx_CT-2A.txt', sep=" ", header=None)
# val_df.columns=['filename', 'label', 'xmin','ymin','xmax','ymax']

# test_df = pd.read_csv('../input/covidxct/test_COVIDx_CT-2A.txt', sep=" ", header=None)
# test_df.columns=['filename', 'label', 'xmin','ymin','xmax','ymax']

metadata_df=pd.read_csv('../input/covidxct/metadata.csv',header=None)
metadata_df.columns=['patient_id','1','2','3','4','5','6','7','8','9']

whole_df=pd.read_csv('../input/whoel-dataframe/out.csv')

In [3]:
metadata_df

patient_id       1        2    3    4          5                 6  \
0         patient id  source  country  sex  age    finding  verified finding   
1               CP_0    CNCB    China  NaN  NaN  Pneumonia               Yes   
2              CP_10    CNCB    China  NaN  NaN  Pneumonia               Yes   
3            CP_1068    CNCB    China  NaN  NaN  Pneumonia               Yes   
4            CP_1070    CNCB    China  NaN  NaN  Pneumonia               Yes   
...              ...     ...      ...  ...  ...        ...               ...   
4497  HUST-Patient93   iCTCF    China    M   69   COVID-19               Yes   
4498  HUST-Patient94   iCTCF    China    F   30   COVID-19               Yes   
4499  HUST-Patient95   iCTCF    China    M   64   COVID-19               Yes   
4500  HUST-Patient96   iCTCF    China    M   32   COVID-19               Yes   
4501  HUST-Patient97   iCTCF    China    M   46   COVID-19               Yes   

                    7      8         9  
0     slice selection   view  modality  
1              Expert  Axial        CT  
2           Automatic  Axial        CT  
3           Automatic  Axial        CT  
4              Expert  Axial        CT  
...               ...    ...       ...  
4497        Automatic  Axial        CT  
4498        Automatic  Axial        CT  
4499        Automatic  Axial        CT  
4500        Automatic  Axial        CT  
4501        Automatic  Axial        CT  

[4502 rows x 10 columns]

In [4]:
# whole_df=pd.concat([train_df,val_df,test_df],axis=0)

In [5]:
# patient_ids=(metadata_df['patient_id']).array
# filenames=(whole_df['filename']).array

In [6]:
def myoperation(filename):
    final_ids=''
    for ids in patient_ids:
        if ids in filename:
            if (len(final_ids)<len(ids)):
                final_ids=ids
    return final_ids

In [7]:
# whole_df['patient_id'] = whole_df.apply(lambda x: myoperation(x['filename']), axis=1)
# compression_opts = dict(method='zip',
#                         archive_name='out.csv')  
# whole_df.to_csv('out.zip', index=False,
#           compression=compression_opts)

In [8]:
whole_df

filename  label  xmin  ymin  xmax  ymax  \
0                  NCP_96_1328_0032.png      2     9    94   512   405   
1                  NCP_96_1328_0035.png      2    10   106   512   405   
2                  NCP_96_1328_0036.png      2    10   105   512   406   
3                  NCP_96_1328_0037.png      2    11   104   512   406   
4                  NCP_96_1328_0038.png      2    11   103   512   406   
...                                 ...    ...   ...   ...   ...   ...   
194917  radiopaedia_29_86491_1-0018.png      2    20   106   622   496   
194918  radiopaedia_29_86491_1-0019.png      2    15   104   624   496   
194919  radiopaedia_29_86491_1-0020.png      2    11   102   626   496   
194920  radiopaedia_29_86491_1-0021.png      2     7   102   630   496   
194921  radiopaedia_29_86491_1-0022.png      2     0   102   630   495   

                  patient_id  
0                     NCP_96  
1                     NCP_96  
2                     NCP_96  
3                     NCP_96  
4                     NCP_96  
...                      ...  
194917  radiopaedia_29_86491  
194918  radiopaedia_29_86491  
194919  radiopaedia_29_86491  
194920  radiopaedia_29_86491  
194921  radiopaedia_29_86491  

[194922 rows x 7 columns]

In [9]:
comp_df=whole_df.groupby(['patient_id'])

In [10]:
comp_df.first()

filename  label  xmin  ymin  \
patient_id                                                                    
CP_0                                  CP_0_3136_0207.png      1    58   149   
CP_10                                CP_10_3153_0172.png      1    30    93   
CP_1068                            CP_1068_3107_0021.png      1     0   113   
CP_1070                            CP_1070_3112_0032.png      1    53   133   
CP_1071                            CP_1071_3114_0035.png      1    33   117   
...                                                  ...    ...   ...   ...   
volume-covid19-A-0695     volume-covid19-A-0695-0013.png      2    57   122   
volume-covid19-A-0696  volume-covid19-A-0696_ct-0016.png      2    80   162   
volume-covid19-A-0697     volume-covid19-A-0697-0020.png      2     0   134   
volume-covid19-A-0698  volume-covid19-A-0698_ct-0000.png      2    75    77   
volume-covid19-A-0699     volume-covid19-A-0699-0021.png      2    23   168   

                       xmax  ymax  
patient_id                         
CP_0                    446   440  
CP_10                   474   419  
CP_1068                 512   388  
CP_1070                 451   391  
CP_1071                 501   451  
...                     ...   ...  
volume-covid19-A-0695   451   444  
volume-covid19-A-0696   423   441  
volume-covid19-A-0697   512   419  
volume-covid19-A-0698   468   373  
volume-covid19-A-0699   485   477  

[3745 rows x 6 columns]

In [11]:
len(comp_df.groups.keys())

3745

In [12]:
print("Any null values : "+str(whole_df.isnull().values.any())+" \nTotal number of null values : "+str(whole_df.isnull().sum().sum()))

Any null values : False 
Total number of null values : 0


In [13]:
print("Only {} patients ct scans are available in the dataset ".format(len(comp_df.groups.keys())))
print("Only {} patients ct scans are available in the dataset ".format(len(whole_df['patient_id'].unique())))

Only 3745 patients ct scans are available in the dataset 
Only 3745 patients ct scans are available in the dataset 


In [14]:
unique_ids=whole_df['patient_id'].unique()
train_size=int(0.8*len(unique_ids))
val_size=int(0.1*len(unique_ids))
test_size=len(unique_ids)-train_size-val_size
unique_ids=shuffle(unique_ids)

train_ids,val_ids,test_ids=unique_ids[0:train_size],unique_ids[train_size:train_size+val_size],unique_ids[train_size+val_size:]

In [15]:
len(train_ids),len(val_ids),len(test_ids)

(2996, 374, 375)

In [16]:
def myoperation1(id):
    if id in train_ids:
        return 'Train'
    if id in val_ids:
        return 'Val'
    if id in test_ids:
        return 'Test'


In [17]:
whole_df['dataset'] = whole_df.apply(lambda x: myoperation1(x['patient_id']), axis=1)

In [18]:
compression_opts = dict(method='zip',
                        archive_name='whole.csv')  
whole_df.to_csv('whole.zip', index=False,
          compression=compression_opts)

In [19]:
train_df=whole_df[whole_df['dataset']=='Train'].copy(deep=True)
val_df=whole_df[whole_df['dataset']=='Val'].copy(deep=True)
test_df=whole_df[whole_df['dataset']=='Test'].copy(deep=True)

In [20]:
print(len(train_df['patient_id'].unique())+len(val_df['patient_id'].unique())+len(test_df['patient_id'].unique()))

3745


In [21]:
compression_opts = dict(method='zip',
                        archive_name='train.csv')  
train_df.to_csv('train.zip', index=False,
          compression=compression_opts)
compression_opts = dict(method='zip',
                        archive_name='val.csv')  
val_df.to_csv('val.zip', index=False,
          compression=compression_opts)
compression_opts = dict(method='zip',
                        archive_name='test.csv')  
test_df.to_csv('test.zip', index=False,
          compression=compression_opts)